In [1]:
from scripts import facebook_model, linkedin_model
import pandas as pd

df = pd.read_csv('data_linkedin/linkedin_data.csv')
text = df['POSTS'][233]
reacts = df['INDEX'][233]

print(reacts)

result = linkedin_model.get_index_prediction(text)
print(result)


1.0780073836122166


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [3]:
df = pd.read_csv('data_facebook/facebook_data.csv')
text = df['POSTS'][233]
reacts = df['ALL_REACTIONS'][233]

print(reacts)

result = facebook_model.get_reactions_prediction(text)
print(result)

19
19


In [11]:
from sklearn.metrics import mean_poisson_deviance
from statsmodels.formula.api import glm
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

list_of_expr = [
    """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY + POSTS_LENGTH + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY + POSTS_LENGTH + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY + POSTS_LENGTH + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY + POSTS_LENGTH + SIMILARITY_TO_BEST + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY + POSTS_LENGTH + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY  + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ POLARITY + POSTS_LENGTH + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES""",
    """ALL_REACTIONS ~ SUBJECTIVITY + POSTS_LENGTH + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES"""
]



iter = 100

for expr in list_of_expr:
    deviance_sum = 0

    for i in range(iter):
        df_facebook = pd.read_csv('data_facebook/facebook_data.csv')

        mask = np.random.rand(len(df_facebook)) < 0.85
        df_train = df_facebook[mask]
        df_test = df_facebook[~mask]
        # print('Training data set length='+str(len(df_train)))
        # print('Testing data set length='+str(len(df_test)))

        # expr = """ALL_REACTIONS ~ POLARITY + SUBJECTIVITY + POSTS_LENGTH + SIMILARITY_TO_BEST + TWENTY_REACTS_BAYES + FIFTY_REACTS_BAYES + HUNDRED_REACTS_BAYES + TWO_HUNDRED_REACTS_BAYES"""

        y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
        y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')
        # print(X_test)

        poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()

        poisson_predictions = poisson_training_results.get_prediction(X_test)
        #summary_frame() returns a pandas DataFrame
        predictions_summary_frame = poisson_predictions.summary_frame()
        #print(predictions_summary_frame)

        predicted_counts=predictions_summary_frame['mean']
        actual_counts = y_test['ALL_REACTIONS']
        # fig = plt.figure()
        # fig.suptitle('Predicted versus actual reactions under a post')
        # predicted, = plt.plot(X_test.index, predicted_counts, 'go-', label='Predicted counts')
        # actual, = plt.plot(X_test.index, actual_counts, 'ro-', label='Actual counts')
        # plt.legend(handles=[predicted, actual])
        # plt.show()

        #Print the training summary.
        # print(poisson_training_results.summary())
        # print(mean_poisson_deviance(actual_counts, predicted_counts))
        deviance_sum += mean_poisson_deviance(actual_counts, predicted_counts)

    print(deviance_sum / iter)

36.310167813549285
38.99913375567249
44.00179473308445
36.632355703794936
38.31551097318221
36.066304864831736
36.21149099244976
37.18568327172651


In [2]:
result = facebook_model.get_mini_model_for_a_text_facebook("Jestem kuba", "Najelpszy tekst")
print(result)

   Intercept  POLARITY  SUBJECTIVITY  POSTS_LENGTH  SIMILARITY_TO_BEST  \
0        1.0       0.0           0.0             2            0.052449   

   TWENTY_REACTS_BAYES  FIFTY_REACTS_BAYES  HUNDRED_REACTS_BAYES  \
0                    0                   0                     0   

   TWO_HUNDRED_REACTS_BAYES  
0                         0  
